In [30]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
coin_list = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'DOGE-USD', 'ADA-USD', 'MATIC-USD', 'DAI-USD', 'WTRX-USD', 'DOT-USD', 'TRX-USD', 'LTC-USD', 'SHIB-USD', 'SOL-USD', 'HEX-USD', 'STETH-USD', 'UNI7083-USD', 'AVAX-USD', 'LEO-USD', 'LINK-USD', 'TON11419-USD']

In [38]:
""" Data to add"""
t1 = 'BTC-USD'
# t2 = 'acn'
t2 = 'DOGE-USD'

stime='2022-12-01'
ftime=None
interval = '1wk'

In [39]:
ticker = yf.Ticker(t2)
resul = ticker.history(start=stime, interval=interval')
resul['Close']


Date
2022-12-01 00:00:00+00:00    0.101584
2022-12-02 00:00:00+00:00    0.102148
2022-12-03 00:00:00+00:00    0.099824
2022-12-04 00:00:00+00:00    0.104286
2022-12-05 00:00:00+00:00    0.101474
2022-12-06 00:00:00+00:00    0.100278
2022-12-07 00:00:00+00:00    0.095783
2022-12-08 00:00:00+00:00    0.098540
2022-12-09 00:00:00+00:00    0.096662
2022-12-10 00:00:00+00:00    0.096457
2022-12-11 00:00:00+00:00    0.092875
2022-12-12 00:00:00+00:00    0.090258
2022-12-13 00:00:00+00:00    0.090940
2022-12-14 00:00:00+00:00    0.088514
2022-12-15 00:00:00+00:00    0.084722
2022-12-16 00:00:00+00:00    0.075942
2022-12-17 00:00:00+00:00    0.079267
2022-12-18 00:00:00+00:00    0.078641
2022-12-19 00:00:00+00:00    0.071429
2022-12-20 00:00:00+00:00    0.074935
2022-12-21 00:00:00+00:00    0.073184
Name: Close, dtype: float64

In [40]:
def ts_1_0(market_ticker, stock_ticker, stime='2022-01-01', ftime=None, interval='1wk'):



    def get_data_from_ticker(tick, stime, ftime, interval):
        ticker = yf.Ticker(tick)
        # df = ticker.history(start='2021-01-01', end='2022-06-30')
        # df = ticker.history(start='2021-01-01', end='2022-07-26')
        # df = ticker.history(start='2020-01-01', end='2021-01-01')
        # df = ticker.history(start='2022-05-28')
        df = ticker.history(start=stime, end=ftime, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    gf = pd.DataFrame(get_data_from_ticker(market_ticker, stime, ftime, interval))
    sf = pd.DataFrame(get_data_from_ticker(stock_ticker, stime, ftime, interval))
    gf[stock_ticker] = sf[stock_ticker]

    def add_percent_update(tick):
        """ date stock's or market's update"""
        gf_copy = gf.copy(deep=True)
        gf_copy = pd.DataFrame(gf_copy.drop(gf.index[0]))
        gf_copy_v = gf_copy[tick].values
        gf_v = gf.copy(deep=True)
        gf_vv = gf_v[tick].values

        for i in range(len(gf_copy)):
            gf_vv[i] = (gf_copy_v[i] - gf_vv[i]) / (gf_vv[i] / 100)

        gf_vv = numpy.insert(gf_vv, 0, 0)
        gf_vv = numpy.delete(gf_vv, -1)
        return gf_vv

    gf[f"% update {market_ticker}"] = add_percent_update(market_ticker)
    gf[f"% update {stock_ticker}"] = add_percent_update(stock_ticker)
    
    def get_index(table):
        gf_copy = table.copy(deep=True)
        gf_copy = gf_copy.reset_index()
        gf_copy = gf_copy.index.tolist()
        return gf_copy

    index = get_index(gf)

    def add_week_difference_market(table, ind):
        """ Sum of date stock's update"""
        gf_copy = table.copy(deep=True)
        gf_copy2 = table.copy(deep=True)
        t = gf_copy2.iloc[:, 2]
        for i in range(len(ind)):
            try:
                if gf_copy.iloc[ind[i], 2] in [gf_copy.iloc[0, 2], gf_copy.iloc[1, 2], gf_copy.iloc[2, 2],
                                               gf_copy.iloc[3, 2]]:
                    t[i] = 0
                else:
                    t[i] = (gf_copy.iloc[ind[i], 2] + gf_copy.iloc[ind[i] - 1, 2] + gf_copy.iloc[ind[i] - 2, 2] +
                            gf_copy.iloc[ind[i] - 3, 2])
            except:
                t[i] = 0
        return t

    gf["Week dif Mar"] = add_week_difference_market(gf, index)

    def add_week_difference_stock(table, ind):
        """ Sum of date stock's update"""
        gf_copy = table.copy(deep=True)
        gf_copy2 = table.copy(deep=True)
        t = gf_copy2.iloc[:, 3]
        for i in range(len(ind)):
            try:
                if gf_copy.iloc[ind[i], 3] in [gf_copy.iloc[0, 3], gf_copy.iloc[1, 3], gf_copy.iloc[2, 3],
                                               gf_copy.iloc[3, 3]]:
                    t[i] = 0
                else:
                    t[i] = (gf_copy.iloc[ind[i], 3] + gf_copy.iloc[ind[i] - 1, 3] + gf_copy.iloc[ind[i] - 2, 3] + gf_copy.iloc[ind[i] - 3, 3])
            except:
                t[i] = 0
        return t

    gf["Week dif St"] = add_week_difference_stock(gf, index)

    return gf

In [43]:
result = ts_1_0(t1, t2, interval='1d')
result

,BTC-USD,DOGE-USD,% update BTC-USD,% update DOGE-USD,Week dif Mar,Week dif St
Date,,,,,,
2022-01-01 00:00:00+00:00,47686.812500,0.173035,0.000000,0.000000,0.000000,0.000000
2022-01-02 00:00:00+00:00,47345.218750,0.174403,-0.716327,0.790592,0.000000,0.000000
2022-01-03 00:00:00+00:00,46458.117188,0.170088,-1.873688,-2.474157,0.000000,0.000000
2022-01-04 00:00:00+00:00,45897.574219,0.168803,-1.206556,-0.755484,0.000000,0.000000
2022-01-05 00:00:00+00:00,43569.003906,0.159420,-5.073406,-5.558555,-8.869977,-7.997603
2022-01-06 00:00:00+00:00,43160.929688,0.160213,-0.936616,0.497425,-9.090265,-8.290770
2022-01-07 00:00:00+00:00,41557.902344,0.155023,-3.714070,-3.239438,-10.930648,-9.056051
2022-01-08 00:00:00+00:00,41733.941406,0.151954,0.423599,-1.979705,-9.300493,-10.280273
2022-01-09 00:00:00+00:00,41911.601562,0.151065,0.425697,-0.585038,-3.801390,-5.306756


In [45]:
np.corrcoef(result[t1].values, result[t2].values)

array([[1.        , 0.86401507],
       [0.86401507, 1.        ]])